In [1]:
import BoolODE as bo
import numpy as np
import pandas as pd
import re

In [68]:
def perturbation(    
    model_definition, 
    simulation_time,
    num_cells,
    sampling_time,
    perturbed_transcription, 
    perturbed_translation,
    output_dir,
    output_filename,
    modeltype
):
    # Normal simulation
    # parameter settings
    gs = bo.GlobalSettings("data", output_dir, True, False, modeltype)
    js1 = bo.JobSettings(
    [{
        "name": model_definition[:-4],
        "model_definition": model_definition,
        "model_initial_conditions": model_definition + "_ics.txt",
        "simulation_time": simulation_time,
        "num_cells": num_cells,
        "do_parallel": True,
        "sample_cells": False,
        "perturbation": False,
        "perturbed_transcription": {},
        "perturbed_translation": {},
        "perturbation_input": "", 
        "perturbation_sampling_time": [],
        "perturbation_sampling_filename": ""
    }]
    )
    # simulation
    boolodejobs = bo.BoolODE(job_settings=js1, global_settings=gs, postproc_settings="")
    boolodejobs.execute_jobs()
    print("Normal simulation ... Done")
    print("Starting perturbation simulation...")
    print(perturbed_transcription)
    
    # Perturbation simulation
    if len(perturbed_transcription) != 0:
        for trans in perturbed_transcription:
            print(trans)
            # job実行
            js2 = bo.JobSettings(
            [{
                "name": model_definition[:-4] + "-perturbation-transcription-" + list(trans.keys())[0],
                "model_definition": model_definition,
                "model_initial_conditions": model_definition + "_ics.txt",
                "simulation_time": simulation_time,
                "num_cells": num_cells,
                "do_parallel": False,
                "sample_cells": False,
                "perturbation": True,
                "perturbed_transcription": trans,#{ 'g1': 10.0 },
                #"perturbed_translation": , # 片方を想定
                "perturbation_input": model_definition[:-4] + "/simulations/", #ここが同じなら初期状態も同じ

                "perturbation_sampling_time": sampling_time,
                "perturbation_sampling_filename": output_dir + "/" + output_filename #ここが同じなら出力も同じ
                # こいつをどこに置くかが問題
            }]
            )
            boolodejobs = bo.BoolODE(job_settings=js2, global_settings=gs, postproc_settings="")
            boolodejobs.execute_jobs()
        E = np.load(output_dir + "/PerturbationSampling-transcription.npy")
        
    else:
        for trans in perturbed_translation:
            print(trans)
            # job実行
            js2 = bo.JobSettings(
            [{
                "name": model_definition[:-4] + "-perturbation-translation-" + list(trans.keys())[0],
                "model_definition": model_definition,
                "model_initial_conditions": model_definition + "_ics.txt",
                "simulation_time": simulation_time,
                "num_cells": num_cells,
                "do_parallel": False,
                "sample_cells": False,
                "perturbation": True,
                "perturbed_transcription": trans,#{ 'g1': 10.0 },
                #"perturbed_translation": , # 片方を想定
                "perturbation_input": model_definition[:-4] + "/simulations/", #ここが同じなら初期状態も同じ

                "perturbation_sampling_time": sampling_time,
                "perturbation_sampling_filename": output_dir + "/" + output_filename #ここが同じなら出力も同じ
                # こいつをどこに置くかが問題
            }]
            )
            boolodejobs = bo.BoolODE(job_settings=js2, global_settings=gs, postproc_settings="")
            boolodejobs.execute_jobs()
        # ToChange    
        E = np.load(output_dir + "/PerturbationSampling-translation.npy")

    
    param_filepath = output_dir + "/" + model_definition[:-4] +  "/parameters.txt"
    with open(param_filepath, 'r') as f:
        # obtain gene names
        df_params = pd.read_csv(param_filepath, sep="\t", skiprows=1, header=None)
        gene_names = list(df_params[df_params[0].str.contains("^n_")][0].str.replace("n_", ""))
        
        # obtaiin GRN edge matrix
        grn_mtx = np.zeros((len(gnames), len(gnames)))
        if modeltype == "hill":
            prefix = "a"
        elif modeltype == "heaviside":
            prefix ="w"
        df_params_prefix = df_params[df_params[0].str.contains("^{}_".format(prefix))]
        df_interactions = df_params_prefix[df_params_prefix[0] == df_params_prefix[0].str.replace("{}_(.*)_(.*)_(.*)".format(prefix), r"{}_\1_\2".format(prefix), regex=True)]
        for i in df_interactions.iterrows():
            interaction = i[1][0]
            val = i[1][1]

            result = re.match("{}_(.*)_(.*)".format(prefix), interaction)
            reg_to = gene_names.index(result.group(1)) 
            reg_from = gene_names.index(result.group(2)) 
            grn_mtx[reg_to, reg_from] = val
        
        
    return E, gene_names, grn_mtx
    
#     if isinstance(trans, float):
#         hoge
#         {gene: hoge} list loop
#   elif isinstance(trans, dict):

In [33]:
import pandas as pd
df_params = pd.read_csv("Synthetic-H/dyn-linear/parameters.txt", sep="\t", skiprows=1, header=None)
gnames = list(df_params[df_params[0].str.contains("^n_")][0].str.replace("n_", ""))
gnames

['g1', 'g2', 'g3', 'g4', 'g5', 'g6', 'g7']

In [30]:
np.zeros((len(gnames), len(gnames)))

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [63]:
gene_names = gnames
grn_mtx = np.zeros((len(gnames), len(gnames)))

In [2]:
df_params_prefix = df_params[df_params[0].str.contains("^w_")]
df_interactions = df_params_prefix[df_params_prefix[0] == df_params_prefix[0].str.replace('w_(.*)_(.*)_(.*)', r'w_\1_\2', regex=True)]
for i in df_interactions.iterrows():
    interaction = i[1][0]
    val = i[1][1]
    
    result = re.match('w_(.*)_(.*)', interaction)
    reg_to = gene_names.index(result.group(1)) 
    reg_from = gene_names.index(result.group(2)) 
    grn_mtx[reg_to, reg_from] = val

NameError: name 'df_params' is not defined

In [67]:
grn_mtx

array([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. , -0.1],
       [ 0.1,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.1,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.1,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0.1,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0.1,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0.1,  0.1]])

In [2]:
# user setting
model_definition = "dyn-linear.txt" #"dyn-cycle.txt" #
perturbed_transcription = [{ 'g1': 10.0 }, { 'g2': 10.0 }]
perturbed_translation = []#[{ 'g3': 5.0 }, { 'g4': 5.0 }]
simulation_time = 9 # * 100 step
num_cells = 20 # = sampling_cells
sampling_time = [100, 300, 500] # 1~simulation_time*100-1?
output_dir = "Synthetic-H"
output_filename = "PerturbationSampling.npy" # npy file
modeltype = "hill" # or heaviside

# default
do_parallel = False
sample_cells = False


# command
# E = perturbation(
#     model_definition, 
#     simulation_time,
#     num_cells,
#     sampling_time,
#     perturbed_transcription, 
#     perturbed_translation,
#     output_dir,
#     output_filename,
#     modeltype
# )

In [3]:
from Perturbation import perturbation as pt

In [4]:
# execution from module
E, gene_names, grn_mtx = pt.perturbation(
    model_definition, 
    simulation_time,
    num_cells,
    sampling_time,
    perturbed_transcription, 
    perturbed_translation,
    output_dir,
    output_filename,
    modeltype
)

  0%|          | 0/20 [00:00<?, ?it/s]

Creating output folders
Starting simulations
Fixing rate parameters to defaults
{'n_g1': 10.0, 'n_g2': 10.0, 'n_g3': 10.0, 'n_g4': 10.0, 'n_g5': 10.0, 'n_g6': 10.0, 'n_g7': 10.0, 'k_g1': 10.0, 'k_g2': 10.0, 'k_g3': 10.0, 'k_g4': 10.0, 'k_g5': 10.0, 'k_g6': 10.0, 'k_g7': 10.0, 'sigmaH_g1': 10.0, 'sigmaH_g2': 10.0, 'sigmaH_g3': 10.0, 'sigmaH_g4': 10.0, 'sigmaH_g5': 10.0, 'sigmaH_g6': 10.0, 'sigmaH_g7': 10.0, 'm_g1': 20.0, 'm_g2': 20.0, 'm_g3': 20.0, 'm_g4': 20.0, 'm_g5': 20.0, 'm_g6': 20.0, 'm_g7': 20.0, 'l_x_g1': 10.0, 'l_x_g2': 10.0, 'l_x_g3': 10.0, 'l_x_g4': 10.0, 'l_x_g5': 10.0, 'l_x_g6': 10.0, 'l_x_g7': 10.0, 'r_g1': 10.0, 'r_g2': 10.0, 'r_g3': 10.0, 'r_g4': 10.0, 'r_g5': 10.0, 'r_g6': 10.0, 'r_g7': 10.0, 'l_p_g1': 1.0, 'l_p_g2': 1.0, 'l_p_g3': 1.0, 'l_p_g4': 1.0, 'l_p_g5': 1.0, 'l_p_g6': 1.0, 'l_p_g7': 1.0}
######################
{'__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis

  5%|▌         | 1/20 [00:00<00:07,  2.63it/s]

[1.0, 10.0, 1.0, 10.0, 1.0, 10.0, 1.0, 10.0, 1.0, 10.0, 1.0, 10.0, 1.0, 10.0]


 10%|█         | 2/20 [00:00<00:06,  2.84it/s]

...

Synthetic-H/dyn-linear/simulations/
[0.04784991695312234, 0.25021863367475805, 0.024119923759348585, 0.22324925919538985, 0.0058811643174068355, 0.03267085573879444, 0.0073183712365201435, 1.9904730948583893, 0.0034592728739978114, 0.07392135047196148, 0.0036554369194768885, 0.09693737632089376, 1.6141480440086038, 17.719815350756203]


 10%|█         | 2/20 [00:00<00:01, 15.61it/s]

Simulations took 7.728 s
starting to concat files


100%|██████████| 20/20 [00:01<00:00, 17.52it/s]


Concating files took 1.14 s
Requested nClusters=1, not performing k-means clustering
Generating input files for pipline...
1. refNetwork
2. PseudoTime.csv
{}
{'g2': 10.0}
1
(3, 7, 20, 7)
Synthetic-H/dyn-linear-PerturbationSampling.npy
Input file generation took 0.16 s
BoolODE.py took 9.10s


In [13]:
type(grn_mtx)

numpy.ndarray

In [6]:
gene_names

['g1', 'g2', 'g3', 'g4', 'g5', 'g6', 'g7']

In [6]:
grn_mtx

array([[ 0.,  0.,  0.,  0.,  0.,  0., -1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  1.]])

In [8]:
len({})

0

In [11]:
E[0,0,:,:]

array([[2.33911183e-03, 3.78427327e-03, 1.26063113e-02, 5.28757987e-03,
        3.98929380e-03, 8.83323447e-03, 2.29430625e+00],
       [1.02218291e-02, 2.96784870e-02, 4.21960216e-02, 7.02293388e-02,
        9.67243507e-03, 7.34848355e-03, 2.44064926e+00],
       [3.84931846e-03, 1.02895643e-02, 1.42831537e-02, 2.91020558e-02,
        3.29358869e-03, 1.25454928e-02, 2.18345325e+00],
       [2.99357644e-02, 8.92061078e-03, 6.91528626e-03, 6.52996128e-02,
        2.73809438e-03, 7.27914942e-03, 2.40234408e+00],
       [2.08682226e-03, 4.34614295e-02, 5.85279112e-03, 3.92359427e-03,
        1.04776582e-03, 7.33614345e-03, 1.87507764e+00],
       [2.09047631e-02, 6.19858396e-04, 5.08922459e-02, 4.44057438e-03,
        4.04191043e-03, 1.57179519e-02, 2.10241825e+00],
       [2.32658233e-02, 3.00280121e-03, 2.21962783e-04, 2.44814431e-03,
        9.56100507e-02, 5.93380286e-03, 2.28657343e+00],
       [3.86800353e-03, 5.22570908e-03, 9.13075139e-03, 1.05550806e-03,
        1.48052129e-03, 3

In [ ]:
# hill model
# Automatically generated by BoolODE
n_g1	10.0
n_g2	10.0
n_g3	10.0
n_g4	10.0
n_g5	10.0
n_g6	10.0
n_g7	10.0
k_g1	10.0
k_g2	10.0
k_g3	10.0
k_g4	10.0
k_g5	10.0
k_g6	10.0
k_g7	10.0
sigmaH_g1	10.0
sigmaH_g2	10.0
sigmaH_g3	10.0
sigmaH_g4	10.0
sigmaH_g5	10.0
sigmaH_g6	10.0
sigmaH_g7	10.0
m_g1	20.0
m_g2	20.0
m_g3	20.0
m_g4	20.0
m_g5	20.0
m_g6	10.0
m_g7	20.0
l_x_g1	10.0
l_x_g2	10.0
l_x_g3	10.0
l_x_g4	10.0
l_x_g5	10.0
l_x_g6	10.0
l_x_g7	10.0
r_g1	10.0
r_g2	10.0
r_g3	10.0
r_g4	10.0
r_g5	10.0
r_g6	10.0
r_g7	10.0
l_p_g1	1.0
l_p_g2	1.0
l_p_g3	1.0
l_p_g4	1.0
l_p_g5	1.0
l_p_g6	1.0
l_p_g7	1.0
alpha_g1	1
alpha_g2	0
alpha_g3	0
alpha_g4	0
alpha_g5	0
alpha_g6	0
alpha_g7	0
a_g1_g7	0
a_g2_g1	1
a_g3_g2	1
a_g4_g3	1
a_g5_g4	1
a_g6_g5	1
a_g7_g6	1
a_g7_g7	1
a_g7_g6_g7	1

In [ ]:
# heaviside model
# Automatically generated by BoolODE
n_g1	10.0
n_g2	10.0
n_g3	10.0
n_g4	10.0
n_g5	10.0
n_g6	10.0
n_g7	10.0
k_g1	10.0
k_g2	10.0
k_g3	10.0
k_g4	10.0
k_g5	10.0
k_g6	10.0
k_g7	10.0
sigmaH_g1	10.0
sigmaH_g2	10.0
sigmaH_g3	10.0
sigmaH_g4	10.0
sigmaH_g5	10.0
sigmaH_g6	10.0
sigmaH_g7	10.0
m_g1	20.0
m_g2	20.0
m_g3	20.0
m_g4	20.0
m_g5	20.0
m_g6	20.0
m_g7	10.0
l_x_g1	10.0
l_x_g2	10.0
l_x_g3	10.0
l_x_g4	10.0
l_x_g5	10.0
l_x_g6	10.0
l_x_g7	10.0
r_g1	10.0
r_g2	10.0
r_g3	10.0
r_g4	10.0
r_g5	10.0
r_g6	10.0
r_g7	10.0
l_p_g1	1.0
l_p_g2	1.0
l_p_g3	1.0
l_p_g4	1.0
l_p_g5	1.0
l_p_g6	1.0
l_p_g7	1.0
omega_g1	1
omega_g2	-1
omega_g3	-1
omega_g4	-1
omega_g5	-1
omega_g6	-1
omega_g7	-1
w_g1_g7	-0.1
w_g2_g1	0.1
w_g3_g2	0.1
w_g4_g3	0.1
w_g5_g4	0.1
w_g6_g5	0.1
w_g7_g6	0.1
w_g7_g7	0.1
w_g7_g6_g7	0.1


## Step by Step simulation

In [2]:
gs = bo.GlobalSettings("data", "Synthetic-H", True, False, "heaviside")
js = bo.JobSettings(
[{
    "name": "dyn-L-1",
    "model_definition": "dyn-linear.txt",
    "model_initial_conditions": "dyn-linear_ics.txt",
    "simulation_time": 9,
    "num_cells": 20,
    "do_parallel": True,
    "sample_cells": False,
    "perturbation": False,
    "perturbed_transcription": { 'g1': 10.0 },
    
    "perturbation_input": ""
}]
)

In [3]:
boolodejobs = bo.BoolODE(job_settings=js, global_settings=gs, postproc_settings="")

In [4]:
boolodejobs.job_settings.jobs

[{'name': 'dyn-L-1',
  'model_definition': 'dyn-linear.txt',
  'model_initial_conditions': 'dyn-linear_ics.txt',
  'simulation_time': 9,
  'num_cells': 20,
  'do_parallel': True,
  'sample_cells': False,
  'perturbation': False,
  'perturbed_transcription': {'g1': 10.0},
  'perturbation_input': ''}]

In [5]:
boolodejobs.execute_jobs()

Creating output folders
Synthetic-H/dyn-L-1 does not exist, creating it...
Starting simulations
Fixing rate parameters to defaults
{'n_g1': 10.0, 'n_g2': 10.0, 'n_g3': 10.0, 'n_g4': 10.0, 'n_g5': 10.0, 'n_g6': 10.0, 'n_g7': 10.0, 'k_g1': 10.0, 'k_g2': 10.0, 'k_g3': 10.0, 'k_g4': 10.0, 'k_g5': 10.0, 'k_g6': 10.0, 'k_g7': 10.0, 'sigmaH_g1': 10.0, 'sigmaH_g2': 10.0, 'sigmaH_g3': 10.0, 'sigmaH_g4': 10.0, 'sigmaH_g5': 10.0, 'sigmaH_g6': 10.0, 'sigmaH_g7': 10.0, 'm_g1': 20.0, 'm_g2': 20.0, 'm_g3': 20.0, 'm_g4': 20.0, 'm_g5': 20.0, 'm_g6': 20.0, 'm_g7': 20.0, 'l_x_g1': 10.0, 'l_x_g2': 10.0, 'l_x_g3': 10.0, 'l_x_g4': 10.0, 'l_x_g5': 10.0, 'l_x_g6': 10.0, 'l_x_g7': 10.0, 'r_g1': 10.0, 'r_g2': 10.0, 'r_g3': 10.0, 'r_g4': 10.0, 'r_g5': 10.0, 'r_g6': 10.0, 'r_g7': 10.0, 'l_p_g1': 1.0, 'l_p_g2': 1.0, 'l_p_g3': 1.0, 'l_p_g4': 1.0, 'l_p_g5': 1.0, 'l_p_g6': 1.0, 'l_p_g7': 1.0}
{'mRNATranscription': 20.0, 'mRNADegradation': 10.0, 'proteinTranslation': 10.0, 'proteinDegradation': 1.0, 'heavisideSigma': 

 20%|██        | 4/20 [00:00<00:00, 30.60it/s]

Simulations took 1.465 s
starting to concat files


100%|██████████| 20/20 [00:00<00:00, 29.90it/s]

Concating files took 0.67 s
Requested nClusters=1, not performing k-means clustering
Generating input files for pipline...
1. refNetwork
2. PseudoTime.csv
Dataset too large.
Sampling 20 cells, one from each simulated trajectory.
Input file generation took 0.09 s
BoolODE.py took 2.25s


In [ ]:
# perturbation loop transcription or translation片方
transcription = [{'g1':5.0}, {'g'}, {}]
translation = [5.0, {}, {}]

In [ ]:
for trans in transcription:
    if isinstance(trans, float):
        hoge
        {gene: hoge} list loop
    else isinstance(trans, dict):
        # job実行

In [6]:
js = bo.JobSettings(
[{
    "name": "dyn-L-1-perturbation",
    "model_definition": "dyn-linear.txt",
    "model_initial_conditions": "dyn-linear_ics.txt",
    "simulation_time": 9,
    "num_cells": 20,
    "do_parallel": True,
    "sample_cells": False,
    "perturbation": True,
    "perturbed_transcription": {},#{ 'g1': 10.0 },
    "perturbed_translation": { 'g1': 5.0 }, # 片方を想定
    "perturbation_input": "dyn-L-1/simulations/", #ここが同じなら初期状態も同じ
    
    "perturbation_sampling_time": [100, 300, 500],
    "perturbation_sampling_filename": "PerturbationSampling.npy" #ここが同じなら出力も同じ
    # こいつをどこに置くかが問題
}]
)

# perturbed_translationの導入
# 複数回のperturbation実験。その際のoutput

In [7]:
boolodejobs = bo.BoolODE(job_settings=js, global_settings=gs, postproc_settings="")

In [8]:
boolodejobs.execute_jobs()

Creating output folders
Starting simulations
Fixing rate parameters to defaults
{'n_g1': 10.0, 'n_g2': 10.0, 'n_g3': 10.0, 'n_g4': 10.0, 'n_g5': 10.0, 'n_g6': 10.0, 'n_g7': 10.0, 'k_g1': 10.0, 'k_g2': 10.0, 'k_g3': 10.0, 'k_g4': 10.0, 'k_g5': 10.0, 'k_g6': 10.0, 'k_g7': 10.0, 'sigmaH_g1': 10.0, 'sigmaH_g2': 10.0, 'sigmaH_g3': 10.0, 'sigmaH_g4': 10.0, 'sigmaH_g5': 10.0, 'sigmaH_g6': 10.0, 'sigmaH_g7': 10.0, 'm_g1': 20.0, 'm_g2': 20.0, 'm_g3': 20.0, 'm_g4': 20.0, 'm_g5': 20.0, 'm_g6': 20.0, 'm_g7': 20.0, 'l_x_g1': 10.0, 'l_x_g2': 10.0, 'l_x_g3': 10.0, 'l_x_g4': 10.0, 'l_x_g5': 10.0, 'l_x_g6': 10.0, 'l_x_g7': 10.0, 'r_g1': 5.0, 'r_g2': 10.0, 'r_g3': 10.0, 'r_g4': 10.0, 'r_g5': 10.0, 'r_g6': 10.0, 'r_g7': 10.0, 'l_p_g1': 1.0, 'l_p_g2': 1.0, 'l_p_g3': 1.0, 'l_p_g4': 1.0, 'l_p_g5': 1.0, 'l_p_g6': 1.0, 'l_p_g7': 1.0}
{'mRNATranscription': 20.0, 'mRNADegradation': 10.0, 'proteinTranslation': 10.0, 'proteinDegradation': 1.0, 'heavisideSigma': 10.0, 'signalingTimescale': 5.0, 'hillCoefficient': 

 15%|█▌        | 3/20 [00:00<00:00, 27.72it/s]

Simulations took 1.461 s
starting to concat files


100%|██████████| 20/20 [00:00<00:00, 28.93it/s]


Concating files took 0.69 s
Requested nClusters=1, not performing k-means clustering
Generating input files for pipline...
1. refNetwork
2. PseudoTime.csv
0
(3, 7, 20, 7)
Input file generation took 0.10 s
BoolODE.py took 2.29s
